# Principal Data Science Task Template

**Author:** Leila Yousefi   
**Date:** 24/07/2025  ({{ today().strftime("%Y-%m-%d") }}
**Objective:** Briefly restate the problem.

## 1. Installations & Imports

## 2. Data pre-processing
### 2.1. load csv file into a dataframe
### 2.2. Summary statistics
### 2.3 Data Quality Checks & Solutions
#### 2.3.1 Validation
#### 2.3.2 Completeness
#### 2.3.3 Uniqueness

## 3. Exploratory Data Analysis
### 3.1 Univariate distributions
### 3.2 Bivariate relationships

## 4. Feature Engineering & Modelling
### 4.1 Train/test split


## 5. Evaluation & Next Steps


In [ ]:
# 1. Installations & Imports: Adjust or add libraries as needed for the task.

# suppress that specific package RuntimeWarning
import warnings
warnings.filterwarnings(
    "ignore",
    category=RuntimeWarning,
    message=".*invalid value encountered in cast.*"
)

# standard libs
import os
import sys
from datetime import datetime

# data libs
import pandas as pd
import numpy as np

# viz libs
import matplotlib.pyplot as plt

# modeling
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# reproducibility
RANDOM_STATE = 42

# Working directory
print("Working directory:", os.getcwd())
print("Notebooks are here:", os.listdir())

# set paths
DATA_DIR = os.path.join("..", "data", "raw")
print("DATA_DIR:", DATA_DIR)
OUTPUT_DIR = os.path.join("..", "data", "processed")
print("OUTPUT_DIR:", OUTPUT_DIR)

In [ ]:
# 2. Data pre-processing: Point the filepaths to data/raw/ and load data.

### 2.1. load csv file into a dataframe
filename = 'pre2018_linked_inv_lpa_data.csv'
df = pd.read_csv(os.path.join(DATA_DIR, filename), low_memory=False)

# Display the first few records
df.head()

### 2.2 Summary statistics & missing values
df.info()
df.describe(include="all")


### 2.3 Data Quality Checks & Solutions:

#### 2.3.1 Validation: **Correct format** 

#### 2.3.2 Completeness: **Decisions on missing data**  
- Column dates → drop rows (where both dates are missing)
- Column X → make derieved id to detect and delete duplicates 
- Column Y → impute median  

#### 2.3.3 Uniqueness: **Decisions onduplicates** 


In [ ]:
#### 2.3.1 Validation: **Correct format** 
# Convert to correct format 
for col in ['registrationdate', 'date_received_in_opg']:
    df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True) # force an out-of-bounds date to NaT, in addition to forcing non-dates (or non-parseable dates) to NaT
# parses dates with the day first, e.g. "10/11/12" is parsed as 2012-11-10, yearfirst=True is not strict, but will prefer to parse with year first.

# Count number of missing records based on missing values in 'registrationdate' 'date_received_in_opg'
n_reg_missing = df['registrationdate'].isna().sum()
n_opg_missing = df['date_received_in_opg'].isna().sum()
print(f"Missing registrationdate: {n_reg_missing}")
print(f"Missing date_received_in_opg: {n_opg_missing}")

# Derive and Define year_month for monthly grouping
df['year_month'] = df['date_received_in_opg'].dt.to_period('M').dt.to_timestamp()
#df['year'] = df['date_received_in_opg'].dt.to_period('Y').dt.to_timestamp()
df['year'] = df['date_received_in_opg'].dt.year
df['month'] = df['date_received_in_opg'].dt.month
df['day'] = df['date_received_in_opg'].dt.day

# Compute delay_days with null assignment for invalid dates
# If registrationdate is NaT or after receipt, delay_days = NaN
df['delay_days'] = (df['date_received_in_opg'] - df['registrationdate']).dt.days
invalid_mask = df['registrationdate'].isna() | (df['registrationdate'] > df['date_received_in_opg'])
df.loc[invalid_mask, 'delay_days'] = pd.NA

# compute “delay in days” and then fill any missing delays with the mean delay for that calendar year 
# (falling back to the overall mean only if an entire year-group is empty):

# Filter out invalid or negative delays
# Keep rows where delay_days is non-negative, drop NaN
df = df[df['delay_days'].notna() & (df['delay_days'] >= 0)].copy()

# Count number of missing records based on missing values in 'delay_days' 
n_delays_missing = df['delay_days'].isna().sum()
print(f"Missing delays: {n_delays_missing}")
delays_missing_ids = df[df['delay_days'].isna()]['case_no']
#print("delays_missing_ids: ", delays_missing_ids)

df['delay_year'] = (
    df['registrationdate'].dt.year
    .fillna(df['date_received_in_opg'].dt.year)
    .astype(int)
)

# Pick a “year” to group on. Use registration‐year if present, otherwise receipt‐year.

# Impute missing delays with the mean for that year
df['delay_days'] = (
    df
    .groupby('delay_year')['delay_days']
    .transform(lambda s: s.fillna(s.mean()))
)

# If an entire year had only missing delays, fill those with the overall mean
overall_mean = df['delay_days'].mean()
df['delay_days'] = df['delay_days'].fillna(overall_mean)

# Count number of missing records based on missing values in 'delay_days' 
n_delays_missing = df['delay_days'].isna().sum()
print(f"Missing delays: {n_delays_missing}")

imputed_delays_days = df[df['case_no'].isin(delays_missing_ids)]['delay_days']
print(f"imputed delays (per day): {imputed_delays_days}")

print(f"imputed df: {df}")

# clean up (Optional) 
df.drop(columns=['delay_year'], inplace=True)

In [ ]:
#### 2.3.2 Completeness: **Decisions on missing data** 
# Missing Data Imputation: Drop rows missing key dates
df = df[df['registrationdate'].notna() & df['date_received_in_opg'].notna()]

#### 2.3.2 Uniqueness: **Decisions onduplicates:**  
# Remove duplicates
# Build hybrid unique ID and remove duplicate
def make_derived_id(row):
    if pd.notna(row['case_no']) and str(row['case_no']).strip():
        return f"{row['case_no']}_{row['date_received_in_opg'].strftime('%Y%m%d')}"
    return str(row['unique_id'])

df['derived_id'] = df.apply(make_derived_id, axis=1)
df = df.drop_duplicates(subset='derived_id')

# Display processed dataframe
print("The first few records:", df.head(5))
print("The last few records:", df.tail(5))

In [ ]:
#### 2.3.4 Accuracy: **measures the correctness of the content of data** 
# Establish which attributes of the data are required and 
# design the logic used to test them based on the business requirement. 
# Consistency is part of Accuracy

In [ ]:
# 3. Exploratory Data Analysis: Insert code cells for plots and summary statistics.


# Define the target variables among the columns
#df["target"] = df['delay_days']
# df["target"] = df["concern_type"]

### 3.1 Univariate distributions
fig, ax = plt.subplots()
df["delay_days"].value_counts().plot(kind="bar", ax=ax)
plt.title("Target distribution")


# ### 3.2 Bivariate relationships
# plt.scatter(df["feature1"], df["feature2"])
# plt.xlabel("feature1")
# plt.ylabel("feature2")
# plt.show()


In [ ]:

# Define periods and concern types
PERIODS = {
    'Pre-pandemic (2016–17)':   (2017, [2016, 2017]),
    'Spike (2018–19)':          (2019, [2018, 2019]),
    'Pandemic (2020–21)':       (2021, [2020, 2021]),
    'Post-pandemic (2022–23)':  (2023, [2022, 2023]),
}
TYPES = ['Financial', 'Health and Welfare', 'Both']

# Compute monthly max and min delay in months
# For each month and concern type, the “worst-case” delay expressed in months:
# So after this step, every row belonging to, say, “Financial” in May 2018 
# will have the same number—the largest delay_days observed among all Financial cases received in May 2018.
# Max monthly delay in months
df['max_delay_months'] = df.groupby(['year_month', 'concern_type'])['delay_days']\
                           .transform('max') / 30.44 # takes each row in a group and 
                                                     # replaces its value with the maximum of that group
                                                    # divide by 30.44 (the average length of a month in days) 
                                                    # to convert that maximum-day figure into “months of delay.”
# Min monthly delay in months
df['min_delay_months'] = df.groupby(['year_month', 'concern_type'])['delay_days']\
                           .transform('min') / 30.44

# Build DataFrame for distributions
records = []
for period, (reg_end, rec_years) in PERIODS.items():
    mask = (
        #(df['registrationdate'].dt.year <= reg_end) &
        df['date_received_in_opg'].dt.year.isin(rec_years) &
        df['concern_type'].isin(TYPES)
    )
    subset = df.loc[mask, ['concern_type', 'delay_days', 'max_delay_months', 'min_delay_months', 'year_month']].copy()
    subset['period'] = period
    records.append(subset)
dist_df = pd.concat(records, ignore_index=True)
dist_df

In [ ]:
df1 = df
df.rename(columns={
    "delay_days": "target"
}, inplace=True)

X = df.drop("target", axis=1)
X

In [ ]:
# 4. Feature Engineering & Modelling: Develop pipelines under the specified headings and record decisions in Markdown.

X = df.drop("target", axis=1)
y = df["target"]


# PCA can’t handle missing values directly. We have two main options:
# Impute the missing values before you scale → PCA
# Drop any rows (or columns) containing NaNs

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# 1. Select numeric columns
num_cols = X.select_dtypes(include=["int64","float64"]).columns

# 2. Impute missing values (here we use the median)
imputer = SimpleImputer(strategy="median")
X_num_imputed = imputer.fit_transform(X[num_cols])

# 3. Scale
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num_imputed)

# 4. PCA to 90% explained variance
pca = PCA(n_components=0.90, random_state=RANDOM_STATE)
X_pca = pca.fit_transform(X_num_scaled)

print(f"PCA reduced {len(num_cols)} → {pca.n_components_} components")

# Or as a single Pipeline
# This is handy if you plan to stick it into a larger Pipeline for CV/reproducibility:
# from sklearn.pipeline import Pipeline

# pca_pipeline = Pipeline([
#     ("imputer", SimpleImputer(strategy="median")),
#     ("scaler", StandardScaler()),
#     ("pca", PCA(n_components=0.90, random_state=RANDOM_STATE)),
# ])

# # fit + transform in one go
# X_pca = pca_pipeline.fit_transform(X[num_cols])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_pca,         # or X_reduced if you use SelectKBest, etc.
    y,             # or y_clean if you dropped rows
    test_size=0.2,
    random_state=RANDOM_STATE
)


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Update your pipelines to force dense output
num_feats = X.select_dtypes(include=["int64","float64"]).columns
cat_feats = X.select_dtypes(include=["object","category"]).columns

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse=False)),  # <-- note sparse=False
])

preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_feats),
    ("cat", cat_pipeline, cat_feats),
    # optional: sparse_threshold=0  to force dense even if some parts remain sparse
], sparse_threshold=0)

# Build your full pipeline
full_pipeline = Pipeline([
    ("prep", preprocessor),
    ("pca", PCA(n_components=0.90, random_state=RANDOM_STATE)),
    ("clf", LogisticRegression(random_state=RANDOM_STATE)),
])

# Cross-validate on the original DataFrame X, Series y
scores = cross_val_score(full_pipeline, X, y, cv=5, scoring="roc_auc")
print("CV AUC:", scores.mean())

In [ ]:
# 5. Evaluation & Next Steps: Clearly report metrics, visualizations, and recommended follow‑up actions.

### 5.1 Final test performance
rf.fit(X_train, y_train)
y_pred = rf.predict_proba(X_test)[:,1]
from sklearn.metrics import roc_auc_score, classification_report
print("Test AUC:", roc_auc_score(y_test, y_pred))
print(classification_report(y_test, rf.predict(X_test)))



### 5.2 Insights & Recommendations
- **Key finding 1:** …
- **Key finding 2:** …
- **Limitations:** data quality, potential biases
- **Next steps:** hyper-parameter tuning, fairness audit, productionize pipeline
